In [79]:
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import minimize
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

In [84]:
columns = ["TotalPT", "VertexChisq", "Isolation"]
signal = np.loadtxt("signal.txt")
background = np.loadtxt("background.txt")
signal = signal[:,4:]
background = background[:,4:]

In [85]:
#Combine the signal and background into one dataset
X = np.concatenate((signal, background))
#Add a label dataset y, which tells the BDT what to aim for (i.e. BDT value should be 1 for signal and 0 for background).
y = np.concatenate((np.ones(signal.shape[0]),
                    np.zeros(background.shape[0])))

In [93]:

X_train,X_test, y_train,y_test = train_test_split(X, y, test_size=0.25, random_state=492)

dt = DecisionTreeClassifier(max_depth=4, min_samples_leaf=int(0.1*len(X)))
bdt = AdaBoostClassifier(dt,
                        algorithm='SAMME',
                        n_estimators=100,
                        learning_rate=0.1)

#Optimise the parameters of the weights using the training sample
bdt.fit(X_train, y_train)

#Test the bdt parameters on the test sample
y_predicted = bdt.predict(X_test)
y_predicted_2 = bdt.predict(X_train)
#This can be used to add the BDT and label columns to your dataset.
df_1 = pd.DataFrame(np.hstack((X_test, y_predicted.reshape(y_predicted.shape[0], -1),y_test.reshape(y_test.shape[0],-1))),
                columns=columns+['BDT val','Label'])
df_2 = pd.DataFrame(np.hstack((X_train, y_predicted_2.reshape(y_predicted_2.shape[0], -1),y_train.reshape(y_train.shape[0],-1))),
                columns=columns+['BDT val','Label'])

df = pd.concat([df_2,df_1],axis=0,ignore_index=True)
print(bdt.score(X_test, y_test))
print(bdt.score(X_train, y_train))
print(df)


0.8218
0.8260666666666666
           TotalPT  VertexChisq  Isolation  BDT val  Label
0      8358.779303    13.252397   0.133740      1.0    0.0
1      2399.897437     6.739473   0.346507      0.0    0.0
2      3374.310418     4.773314   0.149434      0.0    1.0
3      8151.947505     2.859302  -0.224987      1.0    1.0
4      9374.437120    11.706422   0.338918      1.0    1.0
...            ...          ...        ...      ...    ...
19995  6333.991250     4.142610  -0.081129      1.0    0.0
19996  7298.204201     2.130227   0.220424      1.0    1.0
19997  3593.262620     6.549663   0.094788      0.0    0.0
19998  5016.835630    24.192815  -0.467886      0.0    1.0
19999  3136.342096     5.396029   0.043111      0.0    0.0

[20000 rows x 5 columns]


In [96]:
tp = len(df[(df["BDT val"]==1.0)&(df["Label"]==1.0)])
tn = len(df[(df["BDT val"]==0.0)&(df["Label"]==0.0)])
fp = len(df[(df["BDT val"]==0.0)&(df["Label"]==1.0)])
fn = len(df[(df["BDT val"]==1.0)&(df["Label"]==0.0)])

metric = (tp+tn)/(tp+tn+fp+fn)
print(metric)

8362
0.825


In [2]:
background = np.loadtxt("background.txt")
signal = np.loadtxt("signal.txt")

In [20]:
merged = np.concatenate((signal, background))
rows, cols = signal.shape
n = rows
mu = np.mean(merged)
sigma = np.std(merged)
mean_s = np.mean(signal, axis=0)
mean_b = np.mean(background, axis=0)
fisher = n*((mean_s-mu)**2+(mean_b-mu)**2)/sigma**2
print(fisher)


[ 1143.35565056  1164.7431085   3249.63259108  3272.70276118
 94813.227569    3506.062678    3589.58343959]


In [27]:
print(merged.shape)
print(signal.shape)
print(background[0,:])

(20000, 7)
(10000, 7)
[2.14802844e+02 2.97257569e+02 1.05432714e+00 1.58281937e+01
 4.47392260e+03 1.44019512e+01 3.04446727e-01]


In [60]:
index = np.flip(fisher.argsort(axis=0))
print(fisher)
print(index)
np.take_along_axis(fisher, index, axis=0) 


[ 1143.35565056  1164.7431085   3249.63259108  3272.70276118
 94813.227569    3506.062678    3589.58343959]
[4 6 5 3 2 1 0]


array([94813.227569  ,  3589.58343959,  3506.062678  ,  3272.70276118,
        3249.63259108,  1164.7431085 ,  1143.35565056])

In [39]:
np.flip(index)

array([4, 6, 5, 3, 2, 1, 0])

In [90]:
index2 = np.repeat(np.atleast_2d(index),10000, axis=0)
ranked_signal = np.take_along_axis(signal, index2, axis=1)[0]
ranked_background = np.take_along_axis(background, index2, axis=1)[0]

In [89]:
print(background[0,:])
print(np.take_along_axis(background[0,:], index, axis=0))
np.take_along_axis(background, index2, axis=1)[0]

[2.14802844e+02 2.97257569e+02 1.05432714e+00 1.58281937e+01
 4.47392260e+03 1.44019512e+01 3.04446727e-01]
[4.47392260e+03 3.04446727e-01 1.44019512e+01 1.58281937e+01
 1.05432714e+00 2.97257569e+02 2.14802844e+02]


array([4.47392260e+03, 3.04446727e-01, 1.44019512e+01, 1.58281937e+01,
       1.05432714e+00, 2.97257569e+02, 2.14802844e+02])

In [2]:
signal_df = pd.read_csv('signal.txt', sep=" ")
columns = ["PT1", "PT2", "P1", "P2", "TotalPT", "VertexChisq", "Isolation"]
signal_df.columns = columns
background_df = pd.read_csv('background.txt', sep=" ")
background_df.columns = columns



In [12]:
best_three = ['TotalPT','Isolation', 'VertexChisq']
sig = signal_df[best_three]
back = background_df[best_three]
sig.head()
back.head()

,TotalPT,Isolation,VertexChisq
0,4473.922598,0.409195,6.524695
1,4425.499687,-0.074212,10.217255
2,4425.499687,-0.074212,10.478341
3,2526.804360,-0.316457,8.668981
4,2499.779164,0.025940,19.821034


In [57]:
back_max = [back[i].max() for i in back]
back_min = [back[i].min() for i in back]
sig_max = [sig[i].max() for i in sig]
sig_min = [sig[i].min() for i in sig]


mix_max = [max(els) for els in zip(back_max, sig_max)]
mix_min = [min(els) for els in zip(back_min, sig_min)]
d_a = (mix_min[0], mix_max[0])
d_b = d_a
d_c = (mix_min[1], mix_max[1])
d_d = d_c
d_e = (mix_min[2], mix_max[2])
d_f = d_e
bnd = (d_a, d_b, d_c, d_d, d_e, d_f)
print(mix_max)
print(mix_min)
bnd


[35338.679704678325, 0.6209467053413391, 29.997858283815383]
[2010.2144273932568, -2.0, 0.03096386937587567]


((2010.2144273932568, 35338.679704678325),
 (2010.2144273932568, 35338.679704678325),
 (-2.0, 0.6209467053413391),
 (-2.0, 0.6209467053413391),
 (0.03096386937587567, 29.997858283815383),
 (0.03096386937587567, 29.997858283815383))

In [67]:
def minimize_metric(vals):
    a, b, c, d, e, f = vals
    selection_cut = '@a < TotalPT < @b and @c < Isolation < @d and @e < VertexChisq < @f'    
    signal_select = sig.query(selection_cut)
    background_select = back.query(selection_cut)
    signal_efficiency = len(signal_select)*1.0/len(sig)
    background_efficiency = len(background_select)*1.0/len(back)
    TP = len(sig)*signal_efficiency
    FP = len(sig)*(1-signal_efficiency)
    TN = len(back)*(1-background_efficiency)
    FN = len(back)*background_efficiency

    return 1 - (TP+TN)/(TP+FP+TN+FN)

bnds = ((0, None), (0, None))

x0 = [2012, 35000, -1.9, 0.62, 0.031, 29]
res = minimize(minimize_metric, x0, method='nelder-mead', bounds=bnd, tol=1e-6)
print(res)  
print(res.x)          


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 0.26032603260326037
             x: [ 2.925e+03  3.510e+04 -1.924e+00  6.208e-01  3.805e-02
                  9.108e+00]
           nit: 85
          nfev: 303
 final_simplex: (array([[ 2.925e+03,  3.510e+04, ...,  3.805e-02,
                         9.108e+00],
                       [ 2.925e+03,  3.510e+04, ...,  3.805e-02,
                         9.108e+00],
                       ...,
                       [ 2.925e+03,  3.510e+04, ...,  3.805e-02,
                         9.108e+00],
                       [ 2.925e+03,  3.510e+04, ...,  3.805e-02,
                         9.108e+00]]), array([ 2.603e-01,  2.603e-01,  2.603e-01,  2.603e-01,
                        2.603e-01,  2.603e-01,  2.603e-01]))
[ 2.92543394e+03  3.50970451e+04 -1.92431210e+00  6.20846688e-01
  3.80454838e-02  9.10786453e+00]


In [4]:
metric_init = len(signal_df)/(len(background_df)+len(signal_df))
print('Inital metric value is ',metric_init)

Inital metric value is  0.5


In [8]:
selection_cut = 'VertexChisq < 4'

#Reduce the dataset using the query command (for arrays can use array.select)
signal_select = signal_df.query(selection_cut)
background_select = background_df.query(selection_cut)
print(signal_select)
#Calculate the signal and background efficiency using the ratio of DataFrame sizes
signal_efficiency = len(signal_select)*1.0/len(signal_df)
background_efficiency = len(background_select)*1.0/len(background_df)

print('signal efficiency is',signal_efficiency)
print('background efficiency is',background_efficiency)

              PT1          PT2          P1         P2       TotalPT  \
4     1325.279348  1454.354535   30.972987  53.188473  13644.962732   
7      389.882677   200.058017    2.375754   0.104484   5613.125753   
8      171.023727   259.370234    9.540492   4.327980   4363.892633   
14     204.805815   154.881536   14.749565  25.422720   2435.402595   
15    1130.544897   562.524245  153.406243   0.175657  13318.206049   
...           ...          ...         ...        ...           ...   
9985   199.831137   335.838348   14.506075  24.003985   3821.907628   
9987   308.305432   325.982101   22.390938   4.978476   3899.572619   
9988   414.308447   233.456080   88.341328   6.778529   6151.286139   
9991   529.287438   241.172671   18.227750  19.092306   4318.134585   
9993  1125.821133   366.215193   85.367884  23.974989   8955.500707   

      VertexChisq  Isolation  
4        1.455509   0.261870  
7        1.928608  -0.100082  
8        1.518382   0.038937  
14       0.106695  -0.1

In [7]:
TP = len(signal_df)*signal_efficiency
FP = len(signal_df)*(1-signal_efficiency)
TN = len(background_df)*(1-background_efficiency)
FN = len(background_df)*background_efficiency

metric = (TP+TN)/(TP+FP+TN+FN)

print('Metric after selection is',metric)

Metric after selection is 0.6675667566756676
